Seminario 1

Jordi Tudela

URL: https://github.com/bar0net/03MAIR---Algoritmos-de-Optimizacion/blob/master/SEMINARIO/JordiTudela-SE1.ipynb


Ref. Plantilla: https://github.com/raul27868/03MAIR---Algoritmos-de-Optimizacion---2019/blob/master/SEMINARIO/Seminario(plantilla).ipynb

# Combinar cifras y operaciones

In [1]:
import random
cifras = [x for x in range(1,10)]
signos = ['+','-','*','/']

## Espacio de posibilidades

- Cada operación en el espacio de posibilidades tiene 5 cifras (1-9) y 4 signos (+,-,*,/)
- Los números de no pueden repetirse
- Los signos no pueden repetirse
- Asumimos que la división solamente es viable si el resultado tiene resto 0 (división entera exacta)
- Para cifras contemplamos combinaciones sin repetición de 9 elementos elige 5: $\frac{n!}{(n-r)!}=\frac{9!}{(9-5)!}$
- Para signos contemplamos todas las permutaciones posibles de los 4 elementos sin repetición: $n! = 4!$
- **El espacio de posibilidades es:** $\frac{9!·4!}{4!}=9!$

## Resultado máximo (Analítico)

Encontrar el resultado máximo es un caso especial que puede razonarse fácilmente. Para tratar de obtener el resultado máximo podemos separar el problema en 2 segmentos distintos: primero un conjunto que tratamos de maximizar menos (segundo) un conjunto que queremos minimizar.

  1. Max_Resultado = Max_Conjunto - Min_Conjunto
 
- La operación crítica para obtener el resultado máximo es la división, por lo tanto, vamos a fijar que la división va a ser entre 1 para sacar ventaja a su elemento constante.
- Sin el 1, el conjunto mínimo es 2/1 (o 2).
- Para obtener el conjunto máximo partiremos de la multiplicación mayor (9*8) y añadiremos el siguiente valor mayor.

  2. Max_Resultado = 9*8+7-2/1 = 77
  
## Resultado mínimo (Analítico)

Análogamente, podemos utilizar un razonamiento parecido para obtener el resultado mínimo.

  3. Min_Resultado = A + Min_Conjunto - Max_Conjunto
  
Dónde A y Min_Conjunto deben ser lo menor posible y Max_Conjunto debe ser lo mayor posible.

- El Mayor Conjunto posible es 9*8
- Dado que no podemos repetir números, la división menor es 2 (4/2)
- A puede ser el número más pequeño del conjunto de cifras 1

  4. Min_Resultado = 1 + 4/2 - 9*8 = -69

## Fuerza Bruta

In [2]:
# Return all combinations of a collection using DFS strategy
def Combinations(collection, memory=[], depth=None):
    # If collection has one item, return the item in a list
    if len(collection) == 1:
        return [memory + collection]
    
    # If we have reached max depth, return a list 
    # of each item in a list
    if depth != None and depth <= 1:
        return [memory + [x] for x in collection]
    
    output = []
    for i,x in enumerate(collection):
        # Get all combinations of the sublist that doesn't include
        # the current item
        if depth == None:
            step = Combinations(collection[:i] + collection[i+1:], memory + [x])
        else:
            step = Combinations(collection[:i] + collection[i+1:], memory + [x], depth-1)
            
        # Prepend the current item to every combination
        #for y in step:
            #y.insert(0,x)
        output += step
    return output
    
        
def BruteForce(objective, numbers, signs, num_limit = 5, sign_limit = 4):
    sign_combinations = Combinations(signs, depth=sign_limit)
    num_combinations  = Combinations(numbers, depth=num_limit)
    
    for s in sign_combinations:
        for n in num_combinations:
            operation = "{}{}{}{}{}{}{}{}{}".format(n[0], s[0], n[1], s[1], n[2], s[2], n[3], s[3], n[4])

            if eval(operation) == objective:
                return operation
    return None


def BruteForceAll(numbers, signs, num_limit = 5, sign_limit = 4):
    sign_combinations = Combinations(signs, depth=sign_limit)
    num_combinations  = Combinations(numbers, depth=num_limit)
    operations = {}
    
    for s in sign_combinations:
        for n in num_combinations:
            operation = "{}{}{}{}{}{}{}{}{}".format(n[0], s[0], n[1], s[1], n[2], s[2], n[3], s[3], n[4])
            value = eval(operation)
            
            if value%1 != 0:
                continue
            
            if value in operations.keys():
                continue
            operations[value] = operation
    return operations

In [3]:
operations = BruteForceAll(cifras, signos)

In [4]:
min_value = int(min(operations.keys()))
max_value = int(max(operations.keys()))

print("Minimum result:", min_value, "=", operations[min_value])
print("Maximum result: ", max_value, "=", operations[max_value])
print("Contains all integers in-between:", all([x in operations.keys() for x in range(min_value, max_value+1)]))

Minimum result: -69 = 1+4/2-8*9
Maximum result:  77 = 7+8*9-2/1
Contains all integers in-between: True


In [5]:
%timeit BruteForce(random.randint(-69,77), cifras, signos)
sol = BruteForce(random.randint(-69,77), cifras, signos)
print("Test:", eval(sol), "=", sol)

132 ms ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Test: -41.0 = 2+5-6*8/1


El algoritmo de Fuerza Bruta se compone de dos recorridos DFS con orden $O(n!)$ y un doble recorrido para generar todas las combinaciones con orden $O(n^2)$. Así, el algoritmo de Fuerza Bruta es $O(n!)$

## Poda

Como hemos visto en el razonamiento analítico del máximo y el mínimo, este problema nos presenta con, básicamente, una operación con 3 términos:

5. {Resultado = A - D + M} ó {Resultado = A + D - M} 

Podemos acotar el rango de cada uno de los términos:

- Rango(A) = [1,9]
- Rango(D) = [2,9]
- Rango(M) = [2,72]

Para resolver el problema exploraremos las ramas de un arbol con tres niveles tratando de podar prematuramente el mayor número de ramas posible. 

Para ello, en el primer nivel exploraremos el término multiplicativo porqué tiene el rango de valores mayor (de esta forma podemos contener mejor las cotas). 

En segundo lugar, buscaremos términos división viables.

M y D fijan el mayor número de cifras y el signo, así que el último paso es identificar si el término independiente está disponible.

### Complejidad

La complejidad algoritmica del modelo viene dada por las tablas de multiplicación y división (complejidad $O(n^2)$ ya que la poda nos permite eliminar gran parte de los caminos (aunque no es trivial).

Poda:
  - El nivel de multiplicación permite una distancia entre cotas de 15
  - El nivel de división permite una distancia entre cotas de 10
  - La ordenación nos permite llegar a un resultado correcto más rápido (generalmente)
  - **La construcción de las tablas domina la complejidad algoritmica**

In [6]:
def Podar(n, numbers, signs):
    # nos creamos un dict con todas las multiplicaciones posibles
    # y otro dict con todas las divisiones posibles
    
    Mul = {}
    Div = {}
    
    for i,x in enumerate(numbers):
        for y in numbers[i+1:]:
            if x*y in Mul.keys():
                Mul[x*y].append( (x,y) )
            else:
                Mul[x*y] = [(x,y)]
            
            if y%x != 0:
                continue
            
            if y//x in Div.keys():
                Div[y//x].append( (y,x) )
            else:
                Div[y//x] = [(y,x)]
    
    explore = []
    for m, factors in Mul.items():
        for f1,f2 in factors:
            # Conjunto multiplicación positivo
            # La cota inferior será M - max(D) + min(A) = M - 9 + 1 = M-8
            # La cota superior será M - min(D) + max(A) = M - 2 + 9 = M+7
            ci = m - 8
            cs = m + 7

            if n < ci or n > cs:
                continue
            explore.append( (ci, cs, [f1, f2], 1) )
    
        for f1,f2 in factors:
            # Conjunto multiplicación negativo
            # La cota inferior será - M + min(D) + min(A) = - M + 2 + 1 = M+3
            # La cota superior será - M + max(D) + max(A) = - M + 9 + 9 = M+18
            ci = -m + 3
            cs = -m + 18
            
            if n < ci or n > cs:
                continue
            explore.append( (ci, cs, [f1, f2], -1) )

    
    while len(explore) > 0:
        # Reordenamos el conjunto de nodos para priorizar aquellos cuya media
        # entre la cota inferior y la cota superior esté más cerca del objetivo
        explore = sorted(explore, key=lambda x : abs(x[0]+x[1]-2*n) )
        current = explore.pop(0)
        
        # Si el número de elementos almacenados es 2 (los dos factores de la multiplicación)
        # abordamos el segundo nivel del árbol, la división
        if len(current[2]) == 2:
            value = current[3]*current[2][0]*current[2][1]
                
            for d, factors in Div.items():
                for d1, d2 in factors:
                    if d1 in current[2] or d2 in current[2]:
                        continue
                    
                    ci = value - current[3]*d + 1
                    cs = value - current[3]*d + 9
                    
                    if n < ci or n > cs:
                        continue
                        
                    explore.append((ci,cs,[current[2][0], current[2][1], d1, d2], current[3]))
        
        # Si hay 4 elementos en la lista, solamente nos queda determinar el término
        # independiente
        elif len(current[2]) == 4:
            value = current[3]*current[2][0]*current[2][1] - current[3]*current[2][2]/current[2][3] 
            
            x = int(n - value)
            if x not in current[2]:
                return "{}+{}*{}+{}/{}".format(x,current[3]*current[2][0],current[2][1],
                                               -current[3]*current[2][2],current[2][3]).replace("+-","-")
        
    return '0'
    
    
solution = Podar(-30, cifras, signos)
print("Test:", eval(solution), "=", solution)

Test: -30.0 = 6-5*8+4/1


In [7]:
%timeit Podar(random.randint(-69,77), cifras, signos)

63.1 µs ± 1 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
